In [2]:
from future.backports.datetime import datetime
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TKAgg')
from pianoq.lab.mplc.mask_aligner import MPLCAligner
from pianoq.lab.pco_camera import PCOCamera
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mplc_device import MPLCDevice
import os 
import datetime
import time 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
cam = PCOCamera()
cam.set_exposure_time(0.5)

ml = MPLCAligner(use_cam=False)
ml.update('1to5w4f', '5to11w8', pi_steps_x=[0], pi_steps_plane=1)

In [47]:
from pianoq.lab.scripts.live_camera import live_cam
roi=(750, 350, 1200, 850)
live_cam(cam, roi=roi)

In [42]:
roi=(750, 350, 1200, 850)
cam.show_image(roi=roi)

(<Figure size 640x480 with 1 Axes>, <Axes: >)

In [48]:
roi=(750, 350, 1200, 850)
name = 'U1_best.fits'
cam.save_image(path=rf'{dir_path}\{name}', roi=roi)
cam.show_image(roi=roi)

(<Figure size 640x480 with 1 Axes>, <Axes: >)

In [41]:
import glob 
from pianoq.lab.mplc.mask_utils import remove_input_modes
import numpy as np 
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_19_filters_80nm'
masks_path = glob.glob(rf'{dir_path}\U1U*.masks')[0]

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True)
mplc.restore_location()

Got MPLC


In [ ]:
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = rf'{dir_path}\{timestamp}_{name}.cam'
cam.save_image(path)

## motors in place

In [50]:
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

Got Zaber motors
Got Thorlabs motors


In [52]:
mxi.move_absolute(9.107)
myi.move_absolute(3.106)

mxs.move_absolute(11.53)
mys.move_absolute(8.763)

In [86]:
mxi.move_relative(-0.01)

BS
- best idler: ~9K
- best signal: ~8.3K 

Mirror:
- best idler: 
- best signal: 


In [ ]:
# 